In [1]:
from sklearn.datasets import load_iris
import pandas as pd
dataset = pd.read_csv('/home/fernandozagatti/Área de Trabalho/Datasets e treinamento/1.csv')
marvin_initial_dataset = dataset

#Dataset: https://www.kaggle.com/uciml/mushroom-classification

In [2]:
dataset.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
from marvinml.marvinautoml import get_data_preparation
from sklearn.model_selection import train_test_split
import numpy as np

dataset, transformer = get_data_preparation(backend="data_prep", dataframe=dataset, target="class")

X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns='class'), 
            dataset['class'], train_size=0.75, test_size=0.25, random_state=42)

marvin_dataset = {'train_X': X_train, 'train_y': y_train, 'test_X': X_test, 'test_y': y_test, 
                  'transformer': transformer}

Dataset without null values but have categorical data in your set.
Applied techniques:  ['label_encoder'] 

Stage -->  label_encoder


In [4]:
dataset.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,5.0,2.0,4.0,1.0,6.0,1.0,0.0,1.0,4.0,0.0,...,7.0,7.0,0.0,2.0,1.0,4.0,2.0,3.0,5.0,p
1,5.0,2.0,9.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,...,7.0,7.0,0.0,2.0,1.0,4.0,3.0,2.0,1.0,e
2,0.0,2.0,8.0,1.0,3.0,1.0,0.0,0.0,5.0,0.0,...,7.0,7.0,0.0,2.0,1.0,4.0,3.0,2.0,3.0,e
3,5.0,3.0,8.0,1.0,6.0,1.0,0.0,1.0,5.0,0.0,...,7.0,7.0,0.0,2.0,1.0,4.0,2.0,3.0,5.0,p
4,5.0,2.0,3.0,0.0,5.0,1.0,1.0,0.0,4.0,1.0,...,7.0,7.0,0.0,2.0,1.0,0.0,3.0,0.0,1.0,e


In [3]:
from marvinml.marvinautoml import get_automl_classifier,get_automl_regressor
tpot = get_automl_classifier(backend="tpot", generations=5, population_size=20, cv=5, 
                             random_state=42, verbosity=2)
tpot.fit(marvin_dataset['train_X'].copy(), marvin_dataset['train_y'].copy())
marvin_model = tpot.fitted_pipeline_

/home/fernandozagatti/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/fernandozagatti/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/fernandozagatti/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing i

Generation 1 - Current best internal CV score: 1.0
Generation 2 - Current best internal CV score: 1.0
Generation 3 - Current best internal CV score: 1.0
Generation 4 - Current best internal CV score: 1.0
Generation 5 - Current best internal CV score: 1.0

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.1, max_depth=8, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.4)


In [4]:
from sklearn import metrics
predictions = marvin_model.predict(marvin_dataset['test_X'])
marvin_metrics = metrics.accuracy_score(marvin_dataset['test_y'], predictions)

In [5]:
input_message = [['x', 's', 'n', 't', 'p', 'f', 'c', 'n', 'k', 'e', 'e', 's', 's', 'w','w', 'p', 'w', 'o', 'p', 'k', 's', 'u']]

In [6]:
import pandas as pd
from marvinml.marvinautoml import get_data_preparation

input_message = pd.DataFrame(input_message, columns = X_train.columns)
input_message = get_data_preparation(backend="trans", dataframe=input_message, trans=marvin_dataset['transformer'])

In [7]:
input_message

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5.0,2.0,4.0,1.0,6.0,1.0,0.0,1.0,4.0,0.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,2.0,3.0,5.0


In [10]:
# Predictor
final_prediction = marvin_model.predict(input_message)[0]

In [11]:
final_prediction

'p'